⊕ [基本关系模式 - SQLAlchemy 1.3文档](https://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html)


In [1]:
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

## 一对多
一对多关系将外键放在引用父对象的子表上。 relationship()然后在父项上指定，作为引用子项表示的项集合：


In [ ]:
class Parent(Base):
    __tablename__ = 'parent'
    id = Column(Integer, primary_key=True)
    children = relationship("Child")

class Child(Base):
    __tablename__ = 'child'
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey('parent.id'))

In [2]:
# 要在一对多中建立双向关系，其中“反向”侧是多对一，
# 请指定一个附加relationship()并使用relationship.back_populates参数
# 连接两者：

class Parent(Base):
    __tablename__ = 'parent'
    id = Column(Integer, primary_key=True)
    # children = relationship("Child", back_populates="parent")
    children = relationship("Child", backref="parent")

class Child(Base):
    __tablename__ = 'child'
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey('parent.id'))
    parent = relationship("Parent", back_populates="children")

In [1]:
from sqlalchemy import Integer, ForeignKey, String, Column
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()

class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    addresses = relationship("Address", backref="user")

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email = Column(String)
    user_id = Column(Integer, ForeignKey('user.id'))

In [10]:
u1 = User()
a1 = Address()
u1.addresses
print(a1.user)

None


In [11]:
# 一旦将Address其附加到u1.addresses集合，就会填充集合和标量属性：
u1.addresses.append(a1)
print(u1.addresses)
print(a1.user)

In [12]:
# 这种行为也适用于移除操作，以及双方的等效操作。例如，当.user再次设置时None，该Address对象将从反向集合中删除：
a1.user = None
print(u1.addresses)

[]


⊕ [python - 如何构建具有复合主键的表的外键？](https://stackoverrun.com/cn/q/4139106)


In [6]:
import sqlalchemy as db
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
class AreaCode(Base): 
    __tablename__ = 'areacodes' 

    area_code = db.Column(db.Integer, primary_key=True) 


class Exchange(Base): 
    __tablename__ = 'exchanges' 

    exchange = db.Column(db.Integer, primary_key=True) 
    area_code_pk = db.Column(db.Integer, db.ForeignKey('areacodes.area_code'), 
          primary_key=True) 
    area_code = relationship('AreaCode', backref='exchanges') 


class PhoneNumber(Base): 
    __tablename__ = 'phonenumbers' 
    __table_args__ = (
     db.ForeignKeyConstraint(
      ['exchange_exchange', 'exchange_area_code_pk'], 
      ['exchanges.exchange', 'exchanges.area_code_pk'], 
     ), 
    ) 

    phone_number = db.Column(db.Integer, primary_key=True) 
    exchange_exchange = db.Column(db.Integer, primary_key=True) 
    exchange_area_code_pk = db.Column(db.Integer, primary_key=True) 
    exchange = relationship('Exchange', backref='phone_numbers')

⊕ [python - SQLAlchemy multiple foreign keys in one mapped class to the same primary key - Stack Overflow](https://stackoverflow.com/questions/22355890/sqlalchemy-multiple-foreign-keys-in-one-mapped-class-to-the-same-primary-key)


In [8]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship, scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine(u'sqlite:///:memory:', echo=True)
session = scoped_session(sessionmaker(bind=engine))
Base = declarative_base()

#The business case here is that a company can be a stakeholder in another company.
class Company(Base):
    __tablename__ = 'company'
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)

class Stakeholder(Base):
    __tablename__ = 'stakeholder'
    id = Column(Integer, primary_key=True)
    company_id = Column(Integer, ForeignKey('company.id'), nullable=False)
    stakeholder_id = Column(Integer, ForeignKey('company.id'), nullable=False)
    company = relationship("Company", foreign_keys=[company_id])
    stakeholder = relationship("Company", foreign_keys=[stakeholder_id])

Base.metadata.create_all(engine)

# simple query test
q1 = session.query(Company).all()
q2 = session.query(Stakeholder).all()

print(q1)

2019-01-20 17:44:38,813 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-20 17:44:38,814 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 17:44:38,815 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-20 17:44:38,817 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 17:44:38,819 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("company")
2019-01-20 17:44:38,820 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 17:44:38,823 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stakeholder")
2019-01-20 17:44:38,824 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 17:44:38,825 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE company (
	id INTEGER NOT NULL, 
	name VARCHAR(50) NOT NULL, 
	PRIMARY KEY (id)
)


2019-01-20 17:44:38,827 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 17:44:38,829 INFO sqlalchemy.engine.base.Engine COMMIT
2019-01-20 17:44:38,831 INFO sqlalchemy.engine.ba